# Stance Detection for the Fake News Challenge

## Identifying Textual Relationships with Deep Neural Nets

### Check the problem context [here](https://drive.google.com/open?id=1KfWaZyQdGBw8AUTacJ2yY86Yxgw2Xwq0).

### Download files required for the project from [here](https://drive.google.com/open?id=10yf39ifEwVihw4xeJJR60oeFBY30Y5J8).

## Step1: Load the given dataset  

1. Mount the google drive

2. Import Glove embeddings

3. Import the test and train datasets

### Mount the google drive to access required project files

Run the below commands

In [0]:
from google.colab import drive

In [2]:
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


#### Path for Project files on google drive

**Note:** You need to change this path according where you have kept the files in google drive. 

In [0]:
project_path = "/content/drive/My Drive/Data Science/"

### Loading the Glove Embeddings

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'glove.6B.zip', 'r') as z:
  z.extractall()

# Load the dataset [5 Marks]

1. Using [read_csv()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) in pandas load the given train datasets files **`train_bodies.csv`** and **`train_stances.csv`**


2. Using [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) command in pandas merge the two datasets based on the Body ID. 

Note: Save the final merged dataset in a dataframe with name **`dataset`**.

In [0]:
import pandas as pd

In [0]:
train_bodies = pd.read_csv('/content/drive/My Drive/Data Science/train_bodies.csv')
train_stances = pd.read_csv('/content/drive/My Drive/Data Science/train_stances.csv')


In [0]:
dataset = pd.merge(train_bodies , train_stances , on = 'Body ID')

In [9]:
dataset.tail()

,Body ID,articleBody,Headline,Stance
49967,2532,"ANN ARBOR, Mich. – A pizza delivery man in Mic...","Pizza delivery man gets tipped more than $2,00...",agree
49968,2532,"ANN ARBOR, Mich. – A pizza delivery man in Mic...","Pizza delivery man gets $2,000 tip",agree
49969,2532,"ANN ARBOR, Mich. – A pizza delivery man in Mic...","Luckiest Pizza Delivery Guy Ever Gets $2,000 Tip",agree
49970,2532,"ANN ARBOR, Mich. – A pizza delivery man in Mic...",Ann Arbor pizza delivery driver surprised with...,agree
49971,2532,"ANN ARBOR, Mich. – A pizza delivery man in Mic...",Ann Arbor pizza delivery driver surprised with...,agree



<h2> Check1:</h2>
  
<h3> You should see the below output if you run `dataset.head()` command as given below </h3>

In [10]:
dataset.head()

,Body ID,articleBody,Headline,Stance
0,0,A small meteorite crashed into a wooded area i...,"Soldier shot, Parliament locked down after gun...",unrelated
1,0,A small meteorite crashed into a wooded area i...,Tourist dubbed ‘Spider Man’ after spider burro...,unrelated
2,0,A small meteorite crashed into a wooded area i...,Luke Somers 'killed in failed rescue attempt i...,unrelated
3,0,A small meteorite crashed into a wooded area i...,BREAKING: Soldier shot at War Memorial in Ottawa,unrelated
4,0,A small meteorite crashed into a wooded area i...,Giant 8ft 9in catfish weighing 19 stone caught...,unrelated


## Step2: Data Pre-processing and setting some hyper parameters needed for model


#### Run the code given below to set the required parameters.

1. `MAX_SENTS` = Maximum no.of sentences to consider in an article.

2. `MAX_SENT_LENGTH` = Maximum no.of words to consider in a sentence.

3. `MAX_NB_WORDS` = Maximum no.of words in the total vocabualry.

4. `MAX_SENTS_HEADING` = Maximum no.of sentences to consider in a heading of an article.

In [0]:
MAX_NB_WORDS = 20000
MAX_SENTS = 20
MAX_SENTS_HEADING = 1
MAX_SENT_LENGTH = 20
VALIDATION_SPLIT = 0.2

### Download the `Punkt` from nltk using the commands given below. This is for sentence tokenization.

For more info on how to use it, read [this](https://stackoverflow.com/questions/35275001/use-of-punktsentencetokenizer-in-nltk).



In [12]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Tokenizing the text and loading the pre-trained Glove word embeddings for each token  [5 marks] 

Keras provides [Tokenizer API](https://keras.io/preprocessing/text/) for preparing text. Read it before going any further.

#### Import the Tokenizer from keras preprocessing text

In [13]:
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


#### Initialize the Tokenizer class with maximum vocabulary count as `MAX_NB_WORDS` initialized at the start of step2. 

In [0]:
 tk = Tokenizer(MAX_NB_WORDS)

#### Now, using fit_on_texts() from Tokenizer class, lets encode the data 

Note: We need to fit articleBody and Headline also to cover all the words.

In [0]:
codetext = dataset['articleBody'] + dataset['Headline']
tk.fit_on_texts(codetext)

#### fit_on_texts() gives the following attributes in the output as given [here](https://faroit.github.io/keras-docs/1.2.2/preprocessing/text/).

* **word_counts:** dictionary mapping words (str) to the number of times they appeared on during fit. Only set after fit_on_texts was called.

* **word_docs:** dictionary mapping words (str) to the number of documents/texts they appeared on during fit. Only set after fit_on_texts was called.

* **word_index:** dictionary mapping words (str) to their rank/index (int). Only set after fit_on_texts was called.

* **document_count:** int. Number of documents (texts/sequences) the tokenizer was trained on. Only set after fit_on_texts or fit_on_sequences was called.



In [21]:
tk.word_counts

OrderedDict([('a', 956997),
             ('small', 10498),
             ('meteorite', 11621),
             ('crashed', 414),
             ('into', 51208),
             ('wooded', 1108),
             ('area', 14264),
             ('in', 816158),
             ("nicaragua's", 1052),
             ('capital', 13734),
             ('of', 883546),
             ('managua', 3656),
             ('overnight', 1498),
             ('the', 2258230),
             ('government', 47284),
             ('said', 268912),
             ('sunday', 10832),
             ('residents', 6856),
             ('reported', 43852),
             ('hearing', 5028),
             ('mysterious', 1706),
             ('boom', 2918),
             ('that', 468860),
             ('left', 23286),
             ('16', 4344),
             ('foot', 4882),
             ('deep', 2436),
             ('crater', 7366),
             ('near', 25876),
             ("city's", 1452),
             ('airport', 11184),
             ('associated'

In [22]:
tk.word_docs

defaultdict(int,
            {'an': 71648,
             'international': 10213,
             'nicaragua': 2303,
             'press': 13606,
             '2014': 12584,
             'murillo': 1322,
             'diameter': 142,
             'a': 102511,
             'reports': 31591,
             'rock': 1997,
             'all': 39674,
             "city's": 992,
             'streak': 494,
             'foot': 3681,
             'had': 58617,
             'has': 77894,
             'if': 36212,
             'near': 16294,
             'institute': 3272,
             'hearing': 3940,
             'something': 13543,
             'anything': 5532,
             'small': 7352,
             'night': 11548,
             'meteorite': 3745,
             'i': 31314,
             'told': 41129,
             'site': 11484,
             'nicaraguan': 2188,
             'weekend': 6097,
             'understanding': 864,
             'saballos': 692,
             'local': 11399,
             'wa

In [0]:
word_index = tk.word_index

In [26]:

tk.document_count

149916

Now, tokenize the sentences using nltk sent_tokenize() and encode the senteces with the ids we got form the above `t.word_index`

Initialise 2 lists with names `texts` and `articles`.

```
texts = [] to store text of article as it is.

articles = [] split the above text into a list of sentences.
```

In [0]:
texts = []
articles = []

In [0]:
texts = dataset['articleBody']

In [0]:
from nltk.tokenize import sent_tokenize
for items in texts.iteritems(): 
    articles.append(sent_tokenize(items[1]))

## Check 2:

first element of texts and articles should be as given below. 

In [31]:
texts[0]

'A small meteorite crashed into a wooded area in Nicaragua\'s capital of Managua overnight, the government said Sunday. Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city\'s airport, the Associated Press reports. \n\nGovernment spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth." House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports. \nMurillo said Nicaragua will ask international experts to help local scientists in understanding what happened.\n\nThe crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee. He said it is still not clear if the meteorite disintegrated or was buried.\n\nHumbe

In [32]:
articles[0]

["A small meteorite crashed into a wooded area in Nicaragua's capital of Managua overnight, the government said Sunday.",
 "Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city's airport, the Associated Press reports.",
 'Government spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth."',
 'House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports.',
 'Murillo said Nicaragua will ask international experts to help local scientists in understanding what happened.',
 'The crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee.',
 'He said it is still not clear if the meteorite disintegrated or was bu

# Now iterate through each article and each sentence to encode the words into ids using t.word_index  [5 marks] 

Here, to get words from sentence you can use `text_to_word_sequence` from keras preprocessing text.

1. Import text_to_word_sequence

2. Initialize a variable of shape (no.of articles, MAX_SENTS, MAX_SENT_LENGTH) with name `data` with zeros first (you can use numpy [np.zeros](https://docs.scipy.org/doc/numpy/reference/generated/numpy.zeros.html) to initialize with all zeros)and then update it while iterating through the words and sentences in each article.

In [0]:
from keras.preprocessing.text import text_to_word_sequence
import numpy as np

In [0]:
data = np.zeros((len(articles), MAX_SENTS, MAX_SENT_LENGTH)).astype(int)

In [38]:
data.shape

(49972, 20, 20)

In [0]:
for pos1, item in enumerate(articles):
    for pos2, sent in enumerate(item):
        if(sent is not None) & (pos2 < MAX_SENTS):
            temp = text_to_word_sequence(sent)
            for pos3, word in enumerate(temp):
                if (pos3 < MAX_SENT_LENGTH):
                    data[pos1][pos2][pos3] = word_index.get(word, 0)

### Check 3:

Accessing first element in data should give something like given below.

In [41]:
data[0, :, :]

array([[    3,   485,   429,  7208,    81,     3,  3732,   331,     5,
         3888,   350,     4,  1432,  2957,     1,    89,    12,   464,
            0,     0],
       [  758,    95,  1045,     3,  2678,  1750,     7,   188,     3,
         1217,  1075,  2029,   699,   159,     1,  3031,   448,     1,
          555,   239],
       [   89,  1067,  4114,  2346,    12,     3,  1092,  3301,    19,
            1,    89,     2,  1792,     1,   524,  2006,    15,     9,
            3,  3109],
       [  182,  3639,   972,   201,  2555,    43,  6772,  1721,  1252,
            5, 13314, 17927,     1,   778,    31,   736,  3987,    68,
           85,     0],
       [ 2346,    12,  1572,    38,  1094,   351,   777,     2,   367,
          260,  1774,     5,  4448,    70,   494,     0,     0,     0,
            0,     0],
       [    1,   699,   188,    19,     1,   429,    32,     3,  7413,
            4,  2216,  1252,     6,     3,  5268,     4,  1217,  1252,
           12,  3362],
       [  

# Repeat the same process for the `Headings` as well. Use variables with names `texts_heading` and `articles_heading` accordingly. [5 marks] 

In [0]:
texts_headings = []
articles_headings = []
texts_headings = dataset['Headline']
for items in texts_headings.iteritems(): 
    articles_headings.append(sent_tokenize(items[1]))

In [0]:
data_heading = np.zeros((len(articles_headings), MAX_SENTS, MAX_SENT_LENGTH)).astype(int)

In [44]:
data_heading.shape

(49972, 20, 20)

In [0]:
for pos1, item in enumerate(articles_headings):
    for pos2, sent in enumerate(item):
        if(sent is not None) & (pos2 < MAX_SENTS):
            temp = text_to_word_sequence(sent)
            for pos3, word in enumerate(temp):
                if (pos3 < MAX_SENT_LENGTH):
                    data_heading[pos1][pos2][pos3] = word_index.get(word, 0)

In [46]:
data_heading[0, :, :]

array([[  722,   207,   343,  7128,   192,    34,  1335, 11545,    21,
          233,   685,     0,     0,     0,     0,     0,     0,     0,
            0,     0],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0],
       [  

### Now the features are ready, lets make the labels ready for the model to process.

### Convert labels into one-hot vectors

You can use [get_dummies](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html) in pandas to create one-hot vectors.

In [0]:
labels = pd.get_dummies(dataset['Stance'])

In [48]:
labels.sample(5)

,agree,disagree,discuss,unrelated
46694,0,0,1,0
10997,1,0,0,0
19366,0,0,0,1
4570,0,0,0,1
4468,0,0,0,1


### Check 4:

The shape of data and labels shoould match the given below numbers.

In [49]:
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (49972, 20, 20)
Shape of label tensor: (49972, 4)


### Shuffle the data

In [0]:
## get numbers upto no.of articles
indices = np.arange(data.shape[0])
## shuffle the numbers
np.random.shuffle(indices)

In [0]:
## shuffle the data
data = data[indices]
data_heading = data_heading[indices]
## shuffle the labels according to data
labels = labels.iloc[indices]

### Split into train and validation sets. Split the train set 80:20 ratio to get the train and validation sets.


Use the variable names as given below:

x_train, x_val - for body of articles.

x-heading_train, x_heading_val - for heading of articles.

y_train - for training labels.

y_val - for validation labels.



In [0]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

In [53]:
print(x_train.shape)
print(y_train.shape)

print(x_val.shape)
print(y_val.shape)

(39977, 20, 20)
(39977, 4)
(9995, 20, 20)
(9995, 4)


### Check 5:

The shape of x_train, x_val, y_train and y_val should match the below numbers.

In [0]:
print(x_train.shape)
print(y_train.shape)

print(x_val.shape)
print(y_val.shape)

(39978, 20, 20)
(39978, 4)
(9994, 20, 20)
(9994, 4)


In [0]:
x_train = np.reshape(x_train, (39977, 400))
x_val = np.reshape(x_val, (9995, 400))

### Create embedding matrix with the glove embeddings


Run the below code to create embedding_matrix which has all the words and their glove embedding if present in glove word list.

In [59]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('./glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 100))


for word, i in t.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

Loaded 400000 word vectors.


NameError: ignored

# Try the sequential model approach and report the accuracy score. [10 marks]  

### Import layers from Keras to build the model

In [65]:
import tensorflow as tf
from tensorflow import keras
from keras import backend as K
import numpy as np
from keras.layers import LSTM, Dense, Dropout, Embedding, Masking, Bidirectional, GlobalAveragePooling1D
tf.set_random_seed(100)

AttributeError: ignored

### Model

In [62]:
model = keras.Sequential()
model.add(keras.layers.Embedding(len(word_index) + 1, 100))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(64, activation='relu'))

# Dropout for regularization
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(32, activation="relu"))
model.add(keras.layers.Dense(16, activation="relu"))
model.add(keras.layers.Dense(4, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 100)         3367800   
_________________________________________________________________
global_average_pooling1d (Gl (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                6464      
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 6

### Compile and fit the model

In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [64]:
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs = 10, batch_size = 50, verbose = 1)## Train the model

Epoch 1/10
800/800 [==============================] - 29s 36ms/step - loss: 0.8492 - accuracy: 0.7301 - val_loss: 0.8036 - val_accuracy: 0.7262
Epoch 2/10
800/800 [==============================] - 29s 36ms/step - loss: 0.8011 - accuracy: 0.7326 - val_loss: 0.8038 - val_accuracy: 0.7262
Epoch 3/10
800/800 [==============================] - 29s 36ms/step - loss: 0.7969 - accuracy: 0.7326 - val_loss: 0.8044 - val_accuracy: 0.7262
Epoch 4/10
800/800 [==============================] - 28s 35ms/step - loss: 0.7939 - accuracy: 0.7326 - val_loss: 0.8046 - val_accuracy: 0.7262
Epoch 5/10
800/800 [==============================] - 29s 36ms/step - loss: 0.7920 - accuracy: 0.7326 - val_loss: 0.8051 - val_accuracy: 0.7262
Epoch 6/10
800/800 [==============================] - 29s 36ms/step - loss: 0.7895 - accuracy: 0.7326 - val_loss: 0.8057 - val_accuracy: 0.7262
Epoch 7/10
800/800 [==============================] - 29s 36ms/step - loss: 0.7882 - accuracy: 0.7326 - val_loss: 0.8070 - val_accuracy:

## Build the same model with attention layers included for better performance (Optional)

## Fit the model and report the accuracy score for the model with attention layer (Optional)